# MultiClass AdaBoost with ID3 as Base Learner


This notebook demonstrates the implementation of the AdaBoost algorithm using the ID3 decision tree as the base learner. 
The dataset used is the Letter Recognition dataset, where the goal is to classify 26 capital letters based on numerical features.


## Step 1: Load and Preprocess the Data

In [17]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Load the dataset
columns = [
    'letter', 'x-box', 'y-box', 'width', 'high', 'onpix', 'x-bar', 
    'y-bar', 'x2bar', 'y2bar', 'xybar', 'x2ybr', 'xy2br', 
    'x-ege', 'xegvy', 'y-ege', 'yegvx'
]
data = pd.read_csv('letter-recognition.data', header=None, names=columns)

# Map letters to numeric classes for easier processing
data['letter'] = data['letter'].apply(lambda x: ord(x) - ord('A'))

# Separate features and target variable
X = data.iloc[:, 1:]
y = data['letter']

# Split the data into training (80%) and testing (20%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Training set size: {X_train.shape[0]} samples")
print(f"Test set size: {X_test.shape[0]} samples")


Training set size: 16000 samples
Test set size: 4000 samples


## Step 2: Implement the ID3 Decision Tree Algorithm

In [18]:

class ID3DecisionTree:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        self.tree = None

    def _entropy(self, y):
        unique, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        return -np.sum(probabilities * np.log2(probabilities + 1e-9))

    def _information_gain(self, X_column, y, threshold):
        parent_entropy = self._entropy(y)
        left_idx = X_column <= threshold
        right_idx = X_column > threshold
        if len(y[left_idx]) == 0 or len(y[right_idx]) == 0:
            return 0
        n = len(y)
        n_left, n_right = len(y[left_idx]), len(y[right_idx])
        e_left, e_right = self._entropy(y[left_idx]), self._entropy(y[right_idx])
        child_entropy = (n_left / n) * e_left + (n_right / n) * e_right
        return parent_entropy - child_entropy

    def _best_split(self, X, y):
        best_gain = -1
        best_feature = None
        best_threshold = None
        for feature_index in range(X.shape[1]):
            X_column = X[:, feature_index]
            thresholds = np.unique(X_column)
            for threshold in thresholds:
                gain = self._information_gain(X_column, y, threshold)
                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature_index
                    best_threshold = threshold
        return best_feature, best_threshold

    def _build_tree(self, X, y, depth):
        n_samples, n_features = X.shape
        n_labels = len(np.unique(y))
        if depth == self.max_depth or n_labels == 1 or n_samples == 0:
            return np.bincount(y).argmax()
        feature_index, threshold = self._best_split(X, y)
        if feature_index is None:
            return np.bincount(y).argmax()
        left_idxs = X[:, feature_index] <= threshold
        right_idxs = X[:, feature_index] > threshold
        left_subtree = self._build_tree(X[left_idxs], y[left_idxs], depth + 1)
        right_subtree = self._build_tree(X[right_idxs], y[right_idxs], depth + 1)
        return (feature_index, threshold, left_subtree, right_subtree)

    def fit(self, X, y):
        self.tree = self._build_tree(X, y, 0)

    def _traverse_tree(self, x, tree):
        if not isinstance(tree, tuple):
            return tree
        feature_index, threshold, left, right = tree
        if x[feature_index] <= threshold:
            return self._traverse_tree(x, left)
        return self._traverse_tree(x, right)

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.tree) for x in X])


## Step 3: Implement the AdaBoost Algorithm

In [19]:

class MultiClassAdaBoost:
    def __init__(self, base_learner_class, n_estimators=50):
        self.base_learner_class = base_learner_class
        self.n_estimators = n_estimators
        self.models = []
        self.alphas = []
        self.classes = None

    def fit(self, X, y):
        n_samples = X.shape[0]
        self.classes = np.unique(y)
        weights = np.ones(n_samples) / n_samples
        for _ in range(self.n_estimators):
            model = self.base_learner_class(max_depth=5)
            model.fit(X, y)
            y_pred = model.predict(X)
            incorrect = (y_pred != y).astype(int)
            error = np.dot(weights, incorrect) / np.sum(weights)
            if error >= 0.5 or error == 0:
                break
            alpha = 0.5 * np.log((1 - error) / error)
            weights = weights * np.exp(alpha * incorrect)
            weights /= np.sum(weights)
            self.models.append(model)
            self.alphas.append(alpha)

    def predict(self, X):
        class_votes = np.zeros((X.shape[0], len(self.classes)))
        for alpha, model in zip(self.alphas, self.models):
            predictions = model.predict(X)
            for i, pred in enumerate(predictions):
                class_index = np.where(self.classes == pred)[0][0]
                class_votes[i, class_index] += alpha
        return self.classes[np.argmax(class_votes, axis=1)]


## Step 4: Train and Evaluate the Model

In [20]:

from sklearn.metrics import accuracy_score, classification_report

# Train MultiClass AdaBoost
multiclass_adaboost = MultiClassAdaBoost(base_learner_class=ID3DecisionTree, n_estimators=10)
multiclass_adaboost.fit(X_train.values, y_train.values)

# Evaluate on the test set
y_pred = multiclass_adaboost.predict(X_test.values)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")
print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))


Accuracy: 3.95%

Classification Report:

              precision    recall  f1-score   support

           0       0.04      1.00      0.08       158
           1       0.00      0.00      0.00       153
           2       0.00      0.00      0.00       147
           3       0.00      0.00      0.00       161
           4       0.00      0.00      0.00       154
           5       0.00      0.00      0.00       155
           6       0.00      0.00      0.00       155
           7       0.00      0.00      0.00       147
           8       0.00      0.00      0.00       151
           9       0.00      0.00      0.00       149
          10       0.00      0.00      0.00       148
          11       0.00      0.00      0.00       152
          12       0.00      0.00      0.00       158
          13       0.00      0.00      0.00       157
          14       0.00      0.00      0.00       150
          15       0.00      0.00      0.00       161
          16       0.00      0.00      0

c:\Users\sadma\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\sadma\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\sadma\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
